In [1]:
%run -i '../../../utils/setup_environment.ipy'

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import fisher_exact, ranksums, chi2, norm
from statsmodels.sandbox.stats.multicomp import multipletests
import matplotlib.gridspec as gridspec

data_path = '../../../data/'

Setup environment... done!


<span style="color:green">✅ Working on **mskimpact_env** conda environment.</span>

In [2]:
arm_level = pd.read_csv(data_path + 'impact-facets-tp53/new_data_facets/msk_impact_facets_annotated.arm_level.txt', sep='\t')
gene_level = pd.read_csv(data_path + 'impact-facets-tp53/new_data_facets/msk_impact_facets_annotated.gene_level_grepTP53.txt', sep='\t')

In [4]:
# TO CHECK
# Inconsistencies gene_level - arm_level in tp53
from tqdm import tqdm,tqdm_notebook
tqdm_notebook().pandas()
def inconsistency(x):
    if arm_level[arm_level['sample'] == x['sample']][arm_level['arm'] == '17p']['cn_state'].shape[0]>0:
        if arm_level[arm_level['sample'] == x['sample']][arm_level['arm'] == '17p']['cn_state'].values[0] != x.cn_state:
            return True
        else: return False
    else: return False

    

gene_level_tp53 = gene_level[gene_level['gene'] == 'TP53']
gene_level_tp53['inconsistency'] = gene_level_tp53.progress_apply(inconsistency, axis=1)

In [6]:
# We save the inconsistencies
gene_level_tp53.to_pickle(data_path + 'impact-facets-tp53/new_data_facets/gene_level_tp53_inconsistencies.pkl')

In [7]:
import pickle
with open('samples_cohort.pkl', 'rb') as f:
    samples_cohort = pickle.load(f)
    
gene_level_tp53_cohort = gene_level_tp53[gene_level_tp53['sample'].str[:17].isin(samples_cohort)]

In [8]:
get_groupby(gene_level_tp53, 'inconsistency', 'count')

,count
inconsistency,
False,34807
True,7409


In [11]:
gene_level_tp53[gene_level_tp53['inconsistency'] == True][gene_level_tp53['seg_length'] >= 21000000]

,sample,gene,chrom,gene_start,gene_end,tsg,seg,median_cnlr_seg,segclust,seg_start,seg_end,cf.em,tcn.em,lcn.em,cf,tcn,lcn,seg_length,mcn,genes_on_seg,gene_snps,gene_het_snps,spans_segs,cn_state,filter,inconsistency
33,P-0007600-T01-IM5_P-0007600-N01-IM5,TP53,17,7565097,7590868,True,28,-0.206973,4,114669,21490300,0.294082,1,0.000000,0.294082,1,0.000000,21375631,1.000000,349,21,0,False,HETLOSS,PASS,True
44,P-0019444-T01-IM6_P-0019444-N01-IM6,TP53,17,7565097,7590868,True,41,-0.076142,10,3958459,56811700,0.292235,3,1.000000,0.292235,3,1.000000,52853241,2.000000,826,22,5,False,LOSS AFTER,PASS,True
62,P-0034039-T01-IM6_P-0034039-N01-IM6,TP53,17,7565097,7590868,True,45,-0.056943,12,114724,38511700,1.000000,2,1.000000,1.000000,2,1.000000,38396976,1.000000,564,22,0,False,DIPLOID,PASS,True
66,P-0028058-T01-IM6_P-0028058-N01-IM6,TP53,17,7565097,7590868,True,45,-0.092703,7,114669,40500450,1.000000,2,1.000000,1.000000,2,1.000000,40385781,1.000000,661,21,4,False,DIPLOID,PASS,True
96,P-0013185-T01-IM5_P-0013185-N01-IM5,TP53,17,7565097,7590868,True,32,-0.068685,8,114650,38487450,1.000000,2,1.000000,1.000000,2,1.000000,38372800,1.000000,564,21,0,False,DIPLOID,PASS,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71571,P-0050640-T01-IM6_P-0050640-N01-IM6,TP53,17,7565097,7590868,True,45,-0.308850,18,114550,43348750,0.434324,3,1.000000,0.434324,3,1.000000,43234200,2.000000,748,28,1,False,LOSS AFTER,PASS,True
71577,P-0050557-T01-IM6_P-0050557-N01-IM6,TP53,17,7565097,7590868,True,38,-0.187216,6,114600,21536306,0.272170,1,0.000000,0.272170,1,0.000000,21421706,1.000000,349,26,0,False,HETLOSS,PASS,True
71581,P-0049382-T02-IM6_P-0049382-N01-IM6,TP53,17,7565097,7590868,True,21,-0.326716,3,114669,80783150,1.000000,2,1.000000,1.000000,2,1.000000,80668481,1.000000,1203,24,3,False,DIPLOID,PASS,True
71637,P-0050674-T01-IM6_P-0050674-N01-IM6,TP53,17,7565097,7590868,True,32,-0.354240,5,114639,21535775,0.262533,2,0.000000,0.262533,2,0.000000,21421136,2.000000,349,26,0,False,LOSS BEFORE,PASS,True


In [14]:
get_groupby(gene_level_tp53[gene_level_tp53['inconsistency'] == True][gene_level_tp53['seg_length'] >= 21000000], 'cn_state', 'count').sort_values(by='count', ascending=False)

,count
cn_state,
DIPLOID,522
HETLOSS,402
GAIN,264
LOSS AFTER,220
LOSS BEFORE,218
CNLOH,199
TETRAPLOID,177
CNLOH BEFORE & LOSS,158
DOUBLE LOSS AFTER,127


In [78]:
samples_hetloss = set(list(gene_level_tp53[gene_level_tp53['inconsistency'] == True][gene_level_tp53['cn_state'] == 'DIPLOID']['sample']))

get_groupby(arm_level[arm_level['sample'].isin(samples_hetloss)][arm_level['arm'] == '17p'], 'cn_state', 'count')
            
            

,count
cn_state,
AMP,1
CNLOH,84
CNLOH & GAIN,4
CNLOH BEFORE & LOSS,1
DOUBLE LOSS AFTER,93
GAIN,107
HETLOSS,212
HOMDEL,39
LOSS & GAIN,1


In [26]:
samples_hetloss = list(set(gene_level_tp53[gene_level_tp53['inconsistency'] == True][gene_level_tp53['seg_length'] >= 21000000][gene_level_tp53['cn_state'] == 'DIPLOID']['sample']))
samples_het_dip = list(set(arm_level[arm_level['sample'].isin(samples_hetloss)][arm_level['arm'] == '17p'][arm_level['cn_state'] == 'HETLOSS']['sample']))

seg_length = gene_level_tp53[gene_level_tp53['sample'].isin(samples_het_dip)]['seg_length']

pd.set_option('float_format', '{:f}'.format)
pd.DataFrame(seg_length.describe())


,seg_length
count,201.000000
mean,50357676.985075
std,22715870.056944
min,21421637.000000
25%,30916320.000000
50%,38452381.000000
75%,80668381.000000
max,80668800.000000


In [27]:
samples_hetloss[0:3]

['P-0027715-T01-IM6_P-0027715-N01-IM6',
 'P-0006047-T03-IM6_P-0006047-N01-IM6',
 'P-0049301-T02-IM6_P-0049301-N02-IM6']

In [23]:
gene_level_tp53[gene_level_tp53['sample']=='P-0001036-T01-IM3_P-0001036-N01-IM3']

,sample,gene,chrom,gene_start,gene_end,tsg,seg,median_cnlr_seg,segclust,seg_start,seg_end,cf.em,tcn.em,lcn.em,cf,tcn,lcn,seg_length,mcn,genes_on_seg,gene_snps,gene_het_snps,spans_segs,cn_state,filter,inconsistency
18252,P-0001036-T01-IM3_P-0001036-N01-IM3,TP53,17,7565097,7590868,True,44,-0.128134,4,114669,80783009,0.122110,1,0.000000,0.122110,1,0.000000,80668340,1.000000,1203,20,6,False,HETLOSS,PASS,True


In [22]:
arm_level[arm_level['sample'] =='P-0001036-T01-IM3_P-0001036-N01-IM3'][arm_level['arm'] =='17p']

,sample,arm,tcn,lcn,cn_length,arm_length,frac_of_arm,cn_state
414921,P-0001036-T01-IM3_P-0001036-N01-IM3,17p,2,1,22148337,22148337,1.000000,DIPLOID


In [38]:
annotated_cohort = pd.read_csv(data_path + 'impact-facets-tp53/new_data_facets/msk_impact_facets_annotated.cohort.txt', sep='\t')

In [44]:
annotated_cohort[annotated_cohort['sample_id'] == 'P-0049537-T01-IM6_P-0049537-N01-IM6']['wgd'].values[0]

True

In [68]:
gene_level_tp53

,sample,gene,chrom,gene_start,gene_end,tsg,seg,median_cnlr_seg,segclust,seg_start,seg_end,cf.em,tcn.em,lcn.em,cf,tcn,lcn,seg_length,mcn,genes_on_seg,gene_snps,gene_het_snps,spans_segs,cn_state,filter,inconsistency
0,P-0034223-T01-IM6_P-0034223-N01-IM6,TP53,17,7565097,7590868,True,26,-0.178258,5,114669,80783261,1.000000,2,1.0,1.000000,2,1.0,80668592,1.0,1203,27,0,False,DIPLOID,PASS,False
2,P-0009819-T01-IM5_P-0009819-N01-IM5,TP53,17,7565097,7590868,True,21,-0.237684,6,114724,80783024,1.000000,2,1.0,1.000000,2,1.0,80668300,1.0,1203,20,4,False,DIPLOID,PASS,False
3,P-0025956-T01-IM6_P-0025956-N01-IM6,TP53,17,7565097,7590868,True,25,-0.050441,4,114669,80783100,0.169170,3,1.0,0.169170,3,1.0,80668431,2.0,1203,20,0,False,LOSS AFTER,PASS,False
5,P-0027609-T01-IM6_P-0027609-N01-IM6,TP53,17,7565097,7590868,True,27,-0.135182,2,114500,18575692,1.000000,2,1.0,1.000000,2,1.0,18461192,1.0,317,21,4,False,DIPLOID,PASS,False
7,P-0027408-T01-IM6_P-0027408-N01-IM6,TP53,17,7565097,7590868,True,30,-0.130530,6,114450,26370475,0.275073,1,0.0,0.275073,1,0.0,26256025,1.0,359,21,4,False,HETLOSS,PASS,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71757,P-0047736-T01-IM6_P-0047736-N01-IM6,TP53,17,7565097,7590868,True,27,-0.374636,6,114669,80783217,0.184186,2,0.0,0.184186,2,0.0,80668548,2.0,1203,26,5,False,CNLOH,PASS,False
71759,P-0049537-T01-IM6_P-0049537-N01-IM6,TP53,17,7565097,7590868,True,47,-0.493030,5,114629,21544493,0.632827,1,0.0,0.632827,1,0.0,21429864,1.0,349,24,0,False,HETLOSS,PASS,True
71761,P-0047148-T01-IM6_P-0047148-N01-IM6,TP53,17,7565097,7590868,True,28,-0.118122,12,114700,80783100,1.000000,2,1.0,1.000000,2,1.0,80668400,1.0,1203,24,3,False,DIPLOID,PASS,False
71763,P-0049188-T01-IM6_P-0049188-N01-IM6,TP53,17,7565097,7590868,True,55,0.111608,16,114530,7990700,NaN,2,NaN,NaN,2,NaN,7876170,NaN,205,26,1,False,INDETERMINATE,PASS,True
